In [4]:
pip install pandas numpy

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.3 MB 16.3 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 14.0 MB/s eta 0:00:01
     |████████████████████████████████| 346 kB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 508 kB 11.2 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np

In [17]:
df_1801=pd.read_csv('DWC_KXLCLS_OD_DAY_SEOULGU_SEOULGU_201801.csv',encoding='cp949')
df_1801

,배송년월일,송하인_시명,송하인_시코드,송하인_구명,송하인_구코드,수하인_시명,수하인_시코드,수하인_구명,수하인_구코드,대분류_착지물동량 가구/인테리어,대분류_착지물동량 기타,대분류_착지물동량 도서/음반,대분류_착지물동량 디지털/가전,대분류_착지물동량 생활/건강,대분류_착지물동량 스포츠/레저,대분류_착지물동량 식품,대분류_착지물동량 출산/육아,대분류_착지물동량 패션의류,대분류_착지물동량 패션잡화,대분류_착지물동량 화장품/미용
0,20180101,서울특별시,11,용산구,11170,서울특별시,11,서대문구,11410,0,0,0,0,0,8,0,0,0,0,0
1,20180101,서울특별시,11,관악구,11620,서울특별시,11,구로구,11530,0,0,0,0,0,0,0,0,0,0,0
2,20180101,서울특별시,11,관악구,11620,서울특별시,11,성동구,11200,0,0,0,0,0,0,0,0,0,0,0
3,20180101,서울특별시,11,강남구,11680,서울특별시,11,종로구,11110,0,0,10,0,0,0,0,0,0,16,0
4,20180101,서울특별시,11,강남구,11680,서울특별시,11,중구,11140,0,0,5,0,0,0,0,0,0,85,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17936,20180131,서울특별시,11,중구,11140,서울특별시,11,양천구,11470,8,81,10,12,35,8,33,0,98,33,3
17937,20180131,서울특별시,11,중구,11140,서울특별시,11,강서구,11500,15,105,16,20,62,15,25,4,173,41,8
17938,20180131,서울특별시,11,중구,11140,서울특별시,11,구로구,11530,4,91,14,15,37,6,16,0,110,28,6
17939,20180131,서울특별시,11,중구,11140,서울특별시,11,금천구,11545,7,51,16,13,40,7,5,5,80,30,4


In [20]:
df_1801.dtypes

배송년월일                 int64
송하인_시명               object
송하인_시코드               int64
송하인_구명               object
송하인_구코드               int64
수하인_시명               object
수하인_시코드               int64
수하인_구명               object
수하인_구코드               int64
대분류_착지물동량 가구/인테리어     int64
대분류_착지물동량 기타          int64
대분류_착지물동량 도서/음반       int64
대분류_착지물동량 디지털/가전      int64
대분류_착지물동량 생활/건강       int64
대분류_착지물동량 스포츠/레저      int64
대분류_착지물동량 식품          int64
대분류_착지물동량 출산/육아       int64
대분류_착지물동량 패션의류        int64
대분류_착지물동량 패션잡화        int64
대분류_착지물동량 화장품/미용      int64
dtype: object

데이터 병합

In [23]:
import pandas as pd
import glob
import numpy as np
from datetime import datetime

def load_and_merge_data(file_pattern='DWC_KXLCLS_OD_DAY_SEOULGU_SEOULGU_*.csv'): # 파일들을 읽고 병합하는 함수
    all_files = glob.glob(file_pattern)
    df_list = []
    for filename in all_files:
        df = pd.read_csv(filename,encoding='cp949')
        df_list.append(df)
    merged_df = pd.concat(df_list, ignore_index=True)
    merged_df['배송년월일'] = pd.to_datetime(merged_df['배송년월일'], format='%Y%m%d')
    
    return merged_df

def aggregate_daily_logistics(df): #일자별, 출발구-도착구별 총 물동량 집계
    volume_columns = [col for col in df.columns if '대분류_착지물동량' in col]
        # 일자별, 출발구-도착구별 총합 계산
    daily_volume = df.groupby(
        ['배송년월일', '송하인_구명', '수하인_구명']
    )[volume_columns].sum().reset_index()
    daily_volume['총_물동량'] = daily_volume[volume_columns].sum(axis=1)
    
    return daily_volume

def create_adjacency_matrices(daily_volume, unique_districts): #일자별 인접행렬 생성 (GNN용)
    unique_dates = sorted(daily_volume['배송년월일'].unique())
    adj_matrices = {}
    
    for date in unique_dates:
        day_data = daily_volume[daily_volume['배송년월일'] == date]
        
        n = len(unique_districts)
        adj_matrix = np.zeros((n, n))
    
        for _, row in day_data.iterrows():
            i = unique_districts.index(row['송하인_구명'])
            j = unique_districts.index(row['수하인_구명'])
            adj_matrix[i][j] = row['총_물동량']
        
        adj_matrices[date] = adj_matrix
    
    return adj_matrices

def prepare_data_for_analysis(file_pattern='DWC_KXLCLS_OD_DAY_SEOULGU_SEOULGU_*.csv'): # 전체 데이터 준비 프로세스
    merged_df = load_and_merge_data(file_pattern)
    daily_volume = aggregate_daily_logistics(merged_df)
    unique_districts = sorted(list(set(daily_volume['송하인_구명'].unique()) | 
                                 set(daily_volume['수하인_구명'].unique())))
    adj_matrices = create_adjacency_matrices(daily_volume, unique_districts)
    return daily_volume, adj_matrices, unique_districts

In [24]:
# 데이터 준비
daily_volume, adj_matrices, unique_districts = prepare_data_for_analysis()

# 시계열 분석을 위한 피벗 테이블 생성
time_series = daily_volume.pivot_table(
    index='배송년월일',
    values='총_물동량',
    aggfunc='sum'
)

In [25]:
# 데이터 품질 체크
print("결측치 확인:")
print(daily_volume.isnull().sum())

print("\n물동량 기초 통계:")
print(daily_volume['총_물동량'].describe())

# 이상치 확인
for district in unique_districts:
    district_data = daily_volume[
        (daily_volume['송하인_구명'] == district) | 
        (daily_volume['수하인_구명'] == district)
    ]
    print(f"\n{district} 물동량 통계:")
    print(district_data['총_물동량'].describe())

결측치 확인:
배송년월일                0
송하인_구명               0
수하인_구명               0
대분류_착지물동량 가구/인테리어    0
대분류_착지물동량 기타         0
대분류_착지물동량 도서/음반      0
대분류_착지물동량 디지털/가전     0
대분류_착지물동량 생활/건강      0
대분류_착지물동량 스포츠/레저     0
대분류_착지물동량 식품         0
대분류_착지물동량 출산/육아      0
대분류_착지물동량 패션의류       0
대분류_착지물동량 패션잡화       0
대분류_착지물동량 화장품/미용     0
총_물동량                0
dtype: int64

물동량 기초 통계:
count    1.260417e+06
mean     3.535940e+02
std      3.824239e+02
min      0.000000e+00
25%      9.700000e+01
50%      2.630000e+02
75%      4.830000e+02
max      1.908500e+04
Name: 총_물동량, dtype: float64

강남구 물동량 통계:
count    101530.000000
mean        690.816242
std         663.073806
min           0.000000
25%         178.000000
50%         581.000000
75%         974.000000
max       19085.000000
Name: 총_물동량, dtype: float64

강동구 물동량 통계:
count    98537.000000
mean       280.252636
std        262.412920
min          0.000000
25%         87.000000
50%        228.000000
75%        388.000000
max       4435.000000
Name

In [26]:
daily_volume.head()

,배송년월일,송하인_구명,수하인_구명,대분류_착지물동량 가구/인테리어,대분류_착지물동량 기타,대분류_착지물동량 도서/음반,대분류_착지물동량 디지털/가전,대분류_착지물동량 생활/건강,대분류_착지물동량 스포츠/레저,대분류_착지물동량 식품,대분류_착지물동량 출산/육아,대분류_착지물동량 패션의류,대분류_착지물동량 패션잡화,대분류_착지물동량 화장품/미용,총_물동량
0,2018-01-01,강남구,강남구,0,7,28,0,10,0,0,0,8,32,0,85
1,2018-01-01,강남구,강동구,0,6,11,0,4,0,0,0,0,11,0,32
2,2018-01-01,강남구,강북구,0,0,7,0,0,0,0,0,0,0,0,7
3,2018-01-01,강남구,강서구,0,5,45,0,5,0,0,0,9,53,0,117
4,2018-01-01,강남구,관악구,0,0,16,0,4,0,0,0,8,14,0,42


In [33]:
daily_volume.to_csv('총물동량.csv', encoding='cp949')

In [28]:
time_series.head()

,총_물동량
배송년월일,
2018-01-01,6205
2018-01-02,403669
2018-01-03,294872
2018-01-04,249666
2018-01-05,223450


od_matrix: 출발지-도착지 간 총 물동량 매트릭스
district_timeseries: 시계열 형태의 구별 물동량
district_ranking: 구별 물동량 순위
weekday_pattern: 요일별 물동량 패턴
monthly_trend: 월별 물동량 트렌드
origin_volume: 출발지 기준 일별 물동량
destination_volume: 도착지 기준 일별 물동량

In [29]:
import pandas as pd
import numpy as np

def create_analysis_ready_dataframes(daily_volume):
    """
    다양한 분석용 데이터프레임 생성
    """
    # 1. 일자별 구간별 매트릭스 (Origin-Destination Matrix)
    od_matrix = daily_volume.pivot_table(
        index='송하인_구명',
        columns='수하인_구명',
        values='총_물동량',
        aggfunc='sum'
    )
    
    # 2. 시계열 형태의 구별 총 물동량
    district_timeseries = pd.DataFrame()
    
    # 출발지 기준 물동량
    origin_volume = daily_volume.pivot_table(
        index='배송년월일',
        columns='송하인_구명',
        values='총_물동량',
        aggfunc='sum',
        fill_value=0
    )
    district_timeseries['출발_물동량'] = origin_volume.sum(axis=1)
    
    # 도착지 기준 물동량
    destination_volume = daily_volume.pivot_table(
        index='배송년월일',
        columns='수하인_구명',
        values='총_물동량',
        aggfunc='sum',
        fill_value=0
    )
    district_timeseries['도착_물동량'] = destination_volume.sum(axis=1)
    
    # 3. 구별 물동량 순위
    district_ranking = pd.DataFrame({
        '출발_총물동량': daily_volume.groupby('송하인_구명')['총_물동량'].sum(),
        '도착_총물동량': daily_volume.groupby('수하인_구명')['총_물동량'].sum()
    }).round(2)
    
    district_ranking['총_물동량'] = district_ranking['출발_총물동량'] + district_ranking['도착_총물동량']
    district_ranking = district_ranking.sort_values('총_물동량', ascending=False)
    
    # 4. 요일별 물동량 패턴
    daily_volume['요일'] = daily_volume['배송년월일'].dt.day_name()
    weekday_pattern = daily_volume.pivot_table(
        index='요일',
        values='총_물동량',
        aggfunc=['mean', 'std', 'count']
    )
    
    # 5. 월별 물동량 트렌드
    monthly_trend = daily_volume.set_index('배송년월일')['총_물동량'].resample('M').sum()
    
    return {
        'od_matrix': od_matrix,
        'district_timeseries': district_timeseries,
        'district_ranking': district_ranking,
        'weekday_pattern': weekday_pattern,
        'monthly_trend': monthly_trend,
        'origin_volume': origin_volume,
        'destination_volume': destination_volume
    }

def get_district_flow_analysis(daily_volume, district_name):
    """
    특정 구의 물동량 흐름 상세 분석
    """
    # 해당 구의 출발 물동량
    outflow = daily_volume[daily_volume['송하인_구명'] == district_name].copy()
    outflow_summary = outflow.groupby('수하인_구명')['총_물동량'].agg(['sum', 'mean', 'std']).round(2)
    
    # 해당 구의 도착 물동량
    inflow = daily_volume[daily_volume['수하인_구명'] == district_name].copy()
    inflow_summary = inflow.groupby('송하인_구명')['총_물동량'].agg(['sum', 'mean', 'std']).round(2)
    
    return {
        'outflow_summary': outflow_summary.sort_values('sum', ascending=False),
        'inflow_summary': inflow_summary.sort_values('sum', ascending=False)
    }

# 사용 예시
def print_data_summary(daily_volume):
    """
    데이터 요약 정보 출력
    """
    print("=== 데이터 기본 정보 ===")
    print(f"기간: {daily_volume['배송년월일'].min()} ~ {daily_volume['배송년월일'].max()}")
    print(f"총 레코드 수: {len(daily_volume):,}")
    print("\n=== 데이터프레임 구조 ===")
    print(daily_volume.info())
    
    # 변환된 데이터프레임 생성
    analysis_dfs = create_analysis_ready_dataframes(daily_volume)
    
    print("\n=== 구별 총 물동량 순위 (상위 5개 구) ===")
    print(analysis_dfs['district_ranking'].head())
    
    print("\n=== 월별 물동량 트렌드 (최근 5개월) ===")
    print(analysis_dfs['monthly_trend'].tail())
    
    return analysis_dfs

In [30]:
# 데이터 분석 실행
analysis_results = print_data_summary(daily_volume)

# 특정 구의 상세 분석
gangnam_analysis = get_district_flow_analysis(daily_volume, '강남구')
print("\n강남구 물동량 분석:")
print(gangnam_analysis['outflow_summary'].head())

=== 데이터 기본 정보 ===
기간: 2018-01-01 00:00:00 ~ 2023-12-31 00:00:00
총 레코드 수: 1,260,417

=== 데이터프레임 구조 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260417 entries, 0 to 1260416
Data columns (total 15 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   배송년월일              1260417 non-null  datetime64[ns]
 1   송하인_구명             1260417 non-null  object        
 2   수하인_구명             1260417 non-null  object        
 3   대분류_착지물동량 가구/인테리어  1260417 non-null  int64         
 4   대분류_착지물동량 기타       1260417 non-null  int64         
 5   대분류_착지물동량 도서/음반    1260417 non-null  int64         
 6   대분류_착지물동량 디지털/가전   1260417 non-null  int64         
 7   대분류_착지물동량 생활/건강    1260417 non-null  int64         
 8   대분류_착지물동량 스포츠/레저   1260417 non-null  int64         
 9   대분류_착지물동량 식품       1260417 non-null  int64         
 10  대분류_착지물동량 출산/육아    1260417 non-null  int64         
 11  대분류_착지물동량 패션의류     1260417 non-null  in

/var/folders/bf/v7jnwl4x1318f88y9y2nlss40000gn/T/ipykernel_57364/609520023.py:57: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_trend = daily_volume.set_index('배송년월일')['총_물동량'].resample('M').sum()
